In [ ]:
!pip install praw pandas tqdm  # Install Python Reddit API Wrapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 6.8 MB/s eta 0:00:00


In [ ]:
import praw
import pandas as pd
from tqdm import tqdm

# Initialize Reddit instance
reddit = praw.Reddit(
    client_id="ac-UYdXvudwcqAlT5MDF2A",
    client_secret="fQ0ZIGQSpsSZFcb4ScFqaSz6SPwgcQ",
    user_agent="PREDICTDEPRESSION"  # e.g., "DepressionDetectionResearch/1.0"
)

# Verify connection
print(f"Read-only mode: {reddit.read_only}")  # Should be True

Read-only mode: True


In [ ]:
def collect_from_subreddits(subreddits, limit=2000, keywords=None):
    """
    Collect posts from specified subreddits
    """
    data = []
    for sub in subreddits:
        subreddit = reddit.subreddit(sub)

        # Collect top posts (likely to be more substantive)
        for post in tqdm(subreddit.top(limit=limit), desc=f"Collecting from r/{sub}"):
            if keywords and not any(kw.lower() in post.title.lower() for kw in keywords):
                continue

            data.append({
                'id': post.id,
                'title': post.title,
                'text': post.selftext,
                'subreddit': sub,
                'upvotes': post.score,
                'created_utc': post.created_utc,
                'url': post.url,
                'label': 1  # Assuming these are positive cases
            })

    return pd.DataFrame(data)

# Example depression-related subreddits
depression_subs = ['depression', 'SuicideWatch', 'mentalhealth', 'Anxiety']
depression_df = collect_from_subreddits(depression_subs, limit=1000)

# For control/non-depressed data
neutral_subs = ['happy', 'CasualConversation', 'UpliftingNews', 'aww']
neutral_df = collect_from_subreddits(neutral_subs, limit=1000)
neutral_df['label'] = 0  # Negative cases

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

In [ ]:
def search_keywords(keywords, limit=1000, subreddit=None):
    """
    Search Reddit for posts containing specific keywords
    """
    data = []
    for kw in keywords:
        query = f"'{kw}'"
        if subreddit:
            query += f" subreddit:{subreddit}"

        for post in tqdm(reddit.subreddit("all").search(query, limit=limit), desc=f"Searching '{kw}'"):
            data.append({
                'id': post.id,
                'title': post.title,
                'text': post.selftext,
                'subreddit': post.subreddit.display_name,
                'upvotes': post.score,
                'created_utc': post.created_utc,
                'url': post.url,
                'label': 1 if kw in depression_keywords else 0
            })

    return pd.DataFrame(data)

# Example keywords
depression_keywords = ['depressed', 'suicidal', 'hopeless', 'worthless', 'empty inside']
neutral_keywords = ['happy', 'excited', 'joyful', 'grateful', 'content']

depression_search_df = search_keywords(depression_keywords, limit=500)
neutral_search_df = search_keywords(neutral_keywords, limit=500)

Searching 'depressed': 0it [00:00, ?it/s]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

Searching 'depressed': 1it [00:01,  1.78s/it]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

Searching 'depressed': 101it [00:03, 31.98it/s]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynch

In [ ]:
def collect_french_content(limit=1000):
    """
    Collect French-language content from specific subreddits
    """
    french_subs = ['france', 'besoin de parler', 'depression fr']
    keywords_fr = ['déprimé', 'suicide', 'tristesse', 'vide', 'solitude']

    data = []
    for sub in french_subs:
        try:
            subreddit = reddit.subreddit(sub)
            for post in tqdm(subreddit.top(limit=limit), desc=f"Collecting French from r/{sub}"):
                # Simple French content detection
                if any(kw in post.title.lower() or kw in post.selftext.lower() for kw in keywords_fr):
                    data.append({
                        'id': post.id,
                        'title': post.title,
                        'text': post.selftext,
                        'subreddit': sub,
                        'upvotes': post.score,
                        'created_utc': post.created_utc,
                        'url': post.url,
                        'language': 'fr',
                        'label': 1
                    })
                elif any(w in post.title.lower() for w in ['heureux', 'content', 'joie']):
                    data.append({
                        'id': post.id,
                        'title': post.title,
                        'text': post.selftext,
                        'subreddit': sub,
                        'upvotes': post.score,
                        'created_utc': post.created_utc,
                        'url': post.url,
                        'language': 'fr',
                        'label': 0
                    })
        except Exception as e:
            print(f"Error accessing r/{sub}: {e}")

    return pd.DataFrame(data)

french_df = collect_french_content(limit=500)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

Error accessing r/besoin de parler: received 404 HTTP response


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.


Error accessing r/depression fr: received 404 HTTP response


In [ ]:
# Combine all data sources
all_data = pd.concat([
    depression_df,
    neutral_df,
    depression_search_df,
    neutral_search_df,
    french_df
]).sample(frac=1).reset_index(drop=True)

# Clean and prepare data
all_data['combined_text'] = all_data['title'] + " " + all_data['text']
all_data = all_data[all_data['combined_text'].str.len() > 20]  # Remove very short posts

# Save collected data
all_data.to_csv('reddit_depression_data.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load collected data
df = pd.read_csv("reddit_depression_data.csv")

# Drop duplicates and empty posts
df = df.drop_duplicates(subset=["text"])
df = df.dropna(subset=["text"])

# Check class balance (important for depression detection)
print(df["label"].value_counts())

# Split into train/test sets
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"])

label
1    3878
0    1370
Name: count, dtype: int64


In [ ]:
import nltk

# Download all necessary NLTK datasets
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')  # Optional for lemmatization
nltk.download('omw-1.4')  # Required for multilingual support

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def clean_text(text):
    if not isinstance(text, str):
        return ""

    # Remove URLs, special chars, mentions
    text = re.sub(r"http\S+|www\S+|@\w+|#\w+", "", text)
    # Remove non-alphabetic characters (keeping French accents)
    text = re.sub(r"[^a-zA-ZÀ-ÿ\s]", "", text)
    # Convert to lowercase
    text = text.lower()

    # Tokenize and remove stopwords (both English and French)
    try:
        tokens = word_tokenize(text)
        stop_words = set(stopwords.words('english') + stopwords.words('french'))
        text = " ".join([word for word in tokens if word not in stop_words])
    except:
        # Fallback if tokenization fails
        text = " ".join(text.split())

    return text

# Apply cleaning
train_df["cleaned_text"] = train_df["text"].apply(clean_text)
test_df["cleaned_text"] = test_df["text"].apply(clean_text)

In [ ]:
# Sample check
print("Original:", train_df["text"].iloc[0])
print("Cleaned:", train_df["cleaned_text"].iloc[0])

Original: The hardest part of being suicidal? At least for me (19 F), it's looking into the eyes of someone who loves you unconditionally and telling them that you're fine when you're having suicidal thoughts. 

My dad is everything to me. He is the most innocent and caring man I have ever encountered. He has tolerated all my rage attacks, crying episodes, tantrums, and has spoiled and loved me ever since the day I was born. I suffer from bipolar 2 and have been going through a severe depressive episode which usually happens around the same time every year. This year I know exactly what's happening and I want to try my best to get past these thoughts. If not for me, for my dad. I have not known real pain until I saw his eyes water today when he saw me crying. He knows me and what I go through. That hurts me the most.
Cleaned: the hardest part of being suicidal at least for me f its looking into the eyes of someone who loves you unconditionally and telling them that youre fine when your

In [ ]:
!pip install --upgrade transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
g

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch

# 1. Initialize Model and Tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 2. Prepare Dataset (assuming you have train_df and test_df from earlier)
class RedditDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Tokenize your data
train_encodings = tokenizer(train_df["cleaned_text"].tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_df["cleaned_text"].tolist(), truncation=True, padding=True, max_length=128)

# Create datasets
train_dataset = RedditDataset(train_encodings, train_df["label"].tolist())
test_dataset = RedditDataset(test_encodings, test_df["label"].tolist())

# 3. Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",  # Evaluation at the end of each epoch
    save_strategy="epoch",  # Save model at each epoch
    logging_dir="./logs",
    logging_steps=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",                     # Explicitly disable wandb
    no_cuda=False                         # Use GPU if available
)

# 4. Define compute_metrics function
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

# 5. Initialize and Run Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.447700,0.366365,0.815238,0.882567,0.832192,0.939433
2,0.322600,0.342719,0.845714,0.890244,0.938571,0.846649
3,0.198500,0.380349,0.843810,0.891102,0.919178,0.864691


TrainOutput(global_step=789, training_loss=0.3404761654040357, metrics={'train_runtime': 376.062, 'train_samples_per_second': 33.489, 'train_steps_per_second': 2.098, 'total_flos': 828405157800960.0, 'train_loss': 0.3404761654040357, 'epoch': 3.0})

In [ ]:
trainer.save_model("depression_detection_model")

In [ ]:
# Get evaluation metrics on your test set
eval_results = trainer.evaluate(test_dataset)
print("Evaluation Results:")
print(f"- Accuracy: {eval_results['eval_accuracy']:.4f}")
print(f"- F1 Score: {eval_results['eval_f1']:.4f}")
print(f"- Precision: {eval_results['eval_precision']:.4f}")
print(f"- Recall: {eval_results['eval_recall']:.4f}")

Evaluation Results:
- Accuracy: 0.8438
- F1 Score: 0.8911
- Precision: 0.9192
- Recall: 0.8647


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Generate predictions
predictions = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(predictions.predictions), dim=1)

# Classification report
print(classification_report(test_df["label"], preds))

# Confusion matrix
print(confusion_matrix(test_df["label"], preds))

              precision    recall  f1-score   support

           0       0.67      0.78      0.72       274
           1       0.92      0.86      0.89       776

    accuracy                           0.84      1050
   macro avg       0.80      0.82      0.81      1050
weighted avg       0.85      0.84      0.85      1050

[[215  59]
 [105 671]]
